# 🧠 Chunking Strategies Demonstration with UberEats Dataset

This notebook showcases and compares five different text chunking strategies using a real-world dataset document related to UberEats architecture. Chunking is a foundational step in Natural Language Processing (NLP), especially for tasks like embedding, retrieval, summarization, and LLM-based question answering.

We will explore the following techniques:

1. **Fixed-Size Chunking** — Splits the text into equal-sized character blocks.
2. **Recursive Character Text Splitting** — Uses intelligent fallbacks to split text based on structure (paragraphs, lines, sentences).
3. **Semantic Chunking** — Groups semantically related sentences using sentence embeddings.
4. **Language-Based Chunking** — Splits by linguistic units (sentences, paragraphs, or words).
5. **Context-Aware Chunking** — Applies sliding windows with overlap to preserve context across chunks.

Each strategy has trade-offs in terms of **granularity**, **coherence**, and **suitability for downstream tasks** like Retrieval-Augmented Generation (RAG).

The source document is a detailed specification of the UberEats data architecture, ideal for chunking experiments due to its rich structure and vocabulary diversity.

In [1]:
!pip install pdfminer.six langchain sentence-transformers nltk scikit-learn --quiet
import nltk
nltk.download("punkt")


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/luanmorenomaciel/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
from pdfminer.high_level import extract_text

pdf_path = "data/doc-datasets.pdf"
text = extract_text(pdf_path)

print(text[:1000])

Documentação de Datasets: Caso de 
Uso UberEats 

Introdução 

Este documento fornece uma visão geral das entidades de negócio e suas respectivas fontes de 
dados, que serão utilizadas nos laboratórios práticos para desenvolver pipelines de dados 
utilizando Apache Spark e suas APIs. 

Visão Geral da Arquitetura de Dados 

Os dados estão distribuídos em múltiplos sistemas, simulando uma arquitetura típica de 
microserviços: 

●  PostgreSQL: Armazena dados relacionados a motoristas e inventário 
●  MySQL: Mantém informações sobre restaurantes, produtos, avaliações e menu 
●  MongoDB: Contém dados de usuários, itens, recomendações e tickets de suporte 
●  Apache Kafka: Gerencia streams de eventos como pedidos, pagamentos, status, GPS e 

rotas 

 
 
 
 
 
 
 
Entidades de Negócio 

1. Usuários 

Representam os clientes que fazem pedidos na plataforma. 

Fontes: 

●  mongodb/users: Dados principais dos usuários 
●  mssql/users: Informações complementares e profissionais 

Atributos princ

# 📏 **1) Fixed Size Chunking**

This strategy splits the document into fixed-length character chunks, optionally with **overlap**, to preserve partial context across chunk boundaries. It’s a simple and commonly used baseline for many NLP tasks like retrieval and summarization.

### 📌 How it works:
- Divides the text into equal-sized blocks (e.g., 500 characters).
- Optionally includes a fixed number of overlapping characters (e.g., 100) from the previous chunk to **maintain context**.
- Does not consider sentence or paragraph structure.

### ✅ Pros:
- Very simple and fast to implement.
- Useful when uniform chunk size is required (e.g., for token limits).

### ⚠️ Cons:
- Can break sentences and split semantic meaning.
- Overlap helps, but still lacks semantic awareness.

> ✅ Use when you need deterministic chunk sizes and simple logic, especially for testing or early prototypes of Retrieval-Augmented Generation (RAG) pipelines.

In [13]:
def fixed_size_chunking_with_overlap(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - overlap
    return chunks

fixed_chunks = fixed_size_chunking_with_overlap(text, chunk_size=500, overlap=100)
print(f"Total Fixed-Size Chunks with Overlap: {len(fixed_chunks)}")
fixed_chunks[:2]

Total Fixed-Size Chunks with Overlap: 25


['Documentação de Datasets: Caso de \nUso UberEats \n\nIntrodução \n\nEste documento fornece uma visão geral das entidades de negócio e suas respectivas fontes de \ndados, que serão utilizadas nos laboratórios práticos para desenvolver pipelines de dados \nutilizando Apache Spark e suas APIs. \n\nVisão Geral da Arquitetura de Dados \n\nOs dados estão distribuídos em múltiplos sistemas, simulando uma arquitetura típica de \nmicroserviços: \n\n●  PostgreSQL: Armazena dados relacionados a motoristas e inventário \n',
 'a típica de \nmicroserviços: \n\n●  PostgreSQL: Armazena dados relacionados a motoristas e inventário \n●  MySQL: Mantém informações sobre restaurantes, produtos, avaliações e menu \n●  MongoDB: Contém dados de usuários, itens, recomendações e tickets de suporte \n●  Apache Kafka: Gerencia streams de eventos como pedidos, pagamentos, status, GPS e \n\nrotas \n\n \n \n \n \n \n \n \n\x0cEntidades de Negócio \n\n1. Usuários \n\nRepresentam os clientes que fazem pedidos na pla

# 🧩 **2) Recursive Character Text Splitting**

This strategy is smarter than fixed-size chunking — it tries to split the text at semantically meaningful boundaries using a **priority list of separators**, like paragraphs, sentences, spaces, and characters.

It's provided by LangChain’s `RecursiveCharacterTextSplitter`, which is designed for **language models** that benefit from coherent, context-preserving input chunks.

### 📌 How it works:
- Tries to split the text at the largest possible separator (e.g., `\n\n`, `\n`, `.`, `" "`).
- If the resulting chunk is too large, it **recursively uses smaller separators**.
- Supports chunk overlap to preserve context.

### ✅ Pros:
- More natural and semantically meaningful chunks than fixed-size.
- Preserves sentence boundaries where possible.
- Great for long-form documents and LLM pre-processing.

### ⚠️ Cons:
- Slightly more complex than fixed-size.
- Resulting chunk sizes may vary slightly depending on text structure.

> ✅ Use this when you want chunks that maintain coherence without splitting in the middle of important sentences or paragraphs. Ideal for **RAG pipelines**, **summarization**, or **semantic search**.

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def recursive_char_split(text, chunk_size=500, overlap=100):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    return splitter.split_text(text)

recursive_chunks = recursive_char_split(text)
print(f"Total Recursive Chunks: {len(recursive_chunks)}")
recursive_chunks[:2]

Total Recursive Chunks: 28


['Documentação de Datasets: Caso de \nUso UberEats \n\nIntrodução \n\nEste documento fornece uma visão geral das entidades de negócio e suas respectivas fontes de \ndados, que serão utilizadas nos laboratórios práticos para desenvolver pipelines de dados \nutilizando Apache Spark e suas APIs. \n\nVisão Geral da Arquitetura de Dados \n\nOs dados estão distribuídos em múltiplos sistemas, simulando uma arquitetura típica de \nmicroserviços:',
 '●  PostgreSQL: Armazena dados relacionados a motoristas e inventário \n●  MySQL: Mantém informações sobre restaurantes, produtos, avaliações e menu \n●  MongoDB: Contém dados de usuários, itens, recomendações e tickets de suporte \n●  Apache Kafka: Gerencia streams de eventos como pedidos, pagamentos, status, GPS e \n\nrotas \n\n \n \n \n \n \n \n \n\x0cEntidades de Negócio \n\n1. Usuários \n\nRepresentam os clientes que fazem pedidos na plataforma. \n\nFontes:']

# 🧠 **3) Semantic Chunking (with SentenceTransformer)**

This strategy groups sentences based on their **semantic meaning** rather than arbitrary character limits. By leveraging sentence embeddings, we can ensure each chunk holds cohesive and contextually related ideas.

We use `SentenceTransformer` from the `sentence-transformers` library to embed sentences before forming chunks of `N` semantically grouped sentences.

### 📌 How it works:
- Splits the text into individual sentences using `nltk.sent_tokenize()`.
- Embeds each sentence using a transformer model like `all-MiniLM-L6-v2`.
- Groups every `N` sentences into a chunk (default is 5).
- This ensures each chunk has coherent meaning and balanced length.

### ✅ Pros:
- Excellent semantic integrity — sentences in the same chunk are related.
- Great for applications like summarization, document understanding, and RAG.
- Respects natural language boundaries (sentences).

### ⚠️ Cons:
- Slightly heavier on computation (embeddings).
- Chunk size is based on number of sentences, not characters/tokens — may be harder to control model input size.

> ✅ Use this when meaning and topic preservation are more important than strict chunk size control. Excellent for **vector databases**, **semantic search**, and **context-aware LLM prompts**.

In [17]:
import nltk
nltk.download("punkt")

from nltk.tokenize import PunktSentenceTokenizer
from sentence_transformers import SentenceTransformer

punkt_tokenizer = PunktSentenceTokenizer()

def semantic_chunking(text, chunk_size=5):
    sentences = punkt_tokenizer.tokenize(text)
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(sentences, show_progress_bar=True)

    chunks = []
    current_chunk = []

    for sentence in sentences:
        current_chunk.append(sentence)
        if len(current_chunk) >= chunk_size:
            chunks.append(" ".join(current_chunk))
            current_chunk = []

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

semantic_chunks = semantic_chunking(text, chunk_size=5)
print(f"Total Semantic Chunks: {len(semantic_chunks)}")
semantic_chunks[:2]

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/luanmorenomaciel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Batches:   0%|          | 0/2 [00:00<?, ?it/s]/Users/luanmorenomaciel/GitHub/ws-prompt-rag/src/cnk-emb/venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Batches: 100%|██████████| 2/2 [00:02<00:00,  1.39s/it]

Total Semantic Chunks: 12


['Documentação de Datasets: Caso de \nUso UberEats \n\nIntrodução \n\nEste documento fornece uma visão geral das entidades de negócio e suas respectivas fontes de \ndados, que serão utilizadas nos laboratórios práticos para desenvolver pipelines de dados \nutilizando Apache Spark e suas APIs. Visão Geral da Arquitetura de Dados \n\nOs dados estão distribuídos em múltiplos sistemas, simulando uma arquitetura típica de \nmicroserviços: \n\n●  PostgreSQL: Armazena dados relacionados a motoristas e inventário \n●  MySQL: Mantém informações sobre restaurantes, produtos, avaliações e menu \n●  MongoDB: Contém dados de usuários, itens, recomendações e tickets de suporte \n●  Apache Kafka: Gerencia streams de eventos como pedidos, pagamentos, status, GPS e \n\nrotas \n\n \n \n \n \n \n \n \n\x0cEntidades de Negócio \n\n1. Usuários \n\nRepresentam os clientes que fazem pedidos na plataforma. Fontes: \n\n●  mongodb/users: Dados principais dos usuários \n●  mssql/users: Informações complementares

# 🧠 **4) Semantic Chunking (with SentenceTransformer)**

This strategy groups semantically related **sentences** into chunks, preserving meaning more effectively than basic character-based methods.

Instead of splitting by length, we split by **linguistic boundaries (sentences)** and use a **sentence transformer** model to embed the text. In this version, we group every `N` sentences into a chunk — a simplified semantic grouping — but embeddings can be later used for smarter strategies like clustering or similarity-based grouping.

### 📌 How it works:
- Tokenizes the document into sentences using NLTK's `PunktSentenceTokenizer`.
- Embeds each sentence using a transformer model (e.g., `all-MiniLM-L6-v2`).
- Groups every `N` sentences into a single chunk (e.g., 5 sentences per chunk).

### ✅ Pros:
- Preserves semantic structure and sentence integrity.
- Good balance between simplicity and meaning preservation.
- Works well for input into LLMs or vector databases.

### ⚠️ Cons:
- Current logic groups sentences statically by count, not semantic similarity.
- Chunk length may vary in character/token size.

> ✅ Use this when sentence-level integrity and topical coherence are more important than strict token control. It’s ideal for document indexing, semantic search, or long-context LLM input.

🔧 **Next step (optional)**: Enhance this by using cosine similarity of embeddings to create *adaptive* semantic chunks, dynamically grouping similar sentences.

In [19]:
from nltk.tokenize import sent_tokenize, word_tokenize

def language_based_chunking(text, by="paragraph"):
    if by == "sentence":
        return sent_tokenize(text)
    elif by == "word":
        return word_tokenize(text)
    elif by == "paragraph":
        paragraphs = [p.strip() for p in text.split("\n\n") if p.strip()]
        return paragraphs
    else:
        raise ValueError("Unsupported chunk type")

lb_chunks = language_based_chunking(text, by="paragraph")
print(f"Total Language-Based Chunks (Paragraphs): {len(lb_chunks)}")
lb_chunks[:5]

Total Language-Based Chunks (Paragraphs): 134


['Documentação de Datasets: Caso de \nUso UberEats',
 'Introdução',
 'Este documento fornece uma visão geral das entidades de negócio e suas respectivas fontes de \ndados, que serão utilizadas nos laboratórios práticos para desenvolver pipelines de dados \nutilizando Apache Spark e suas APIs.',
 'Visão Geral da Arquitetura de Dados',
 'Os dados estão distribuídos em múltiplos sistemas, simulando uma arquitetura típica de \nmicroserviços:']

# 🧵 **5) Context-Aware Chunking (Sliding Window / Stride)**

This strategy splits the text using a **sliding window** approach, where each chunk includes some content from the previous chunk. It is designed to **preserve context** across boundaries by adding overlap — ensuring smoother continuity between chunks.

This is particularly useful for **long-form document processing**, where cutting off context between chunks can lead to degraded performance in tasks like summarization, retrieval, or LLM question answering.

### 📌 How it works:
- Text is split into words or tokens.
- A **window of fixed length** (e.g., 600 words) is extracted.
- The window then **slides forward by a smaller step** (e.g., 100 words).
- The result: overlapping chunks that maintain forward and backward context.

### ✅ Pros:
- Maintains flow of thought between chunks.
- Avoids hard cuts that break meaning.
- Works great with models like GPT, Claude, Gemini for context-aware tasks.

### ⚠️ Cons:
- Produces **more chunks**, increasing memory/compute cost.
- Chunks may include **redundant or repeated content**.

> ✅ Use this when downstream tasks benefit from maintaining the full context, such as in **retrieval-augmented generation (RAG)**, **dialogue agents**, or **multi-turn summarization**.

In [20]:
def context_aware_chunking(text, max_chunk_length=600, stride=100):
    words = text.split()
    chunks = []
    for i in range(0, len(words), stride):
        chunk = words[i:i + max_chunk_length]
        if chunk:
            chunks.append(" ".join(chunk))
    return chunks

context_chunks = context_aware_chunking(text)
print(f"Total Context-Aware Chunks: {len(context_chunks)}")
context_chunks[:2]

Total Context-Aware Chunks: 13


['Documentação de Datasets: Caso de Uso UberEats Introdução Este documento fornece uma visão geral das entidades de negócio e suas respectivas fontes de dados, que serão utilizadas nos laboratórios práticos para desenvolver pipelines de dados utilizando Apache Spark e suas APIs. Visão Geral da Arquitetura de Dados Os dados estão distribuídos em múltiplos sistemas, simulando uma arquitetura típica de microserviços: ● PostgreSQL: Armazena dados relacionados a motoristas e inventário ● MySQL: Mantém informações sobre restaurantes, produtos, avaliações e menu ● MongoDB: Contém dados de usuários, itens, recomendações e tickets de suporte ● Apache Kafka: Gerencia streams de eventos como pedidos, pagamentos, status, GPS e rotas Entidades de Negócio 1. Usuários Representam os clientes que fazem pedidos na plataforma. Fontes: ● mongodb/users: Dados principais dos usuários ● mssql/users: Informações complementares e profissionais Atributos principais: ● user_id: Identificador único do usuário ● 